<a href="https://colab.research.google.com/github/stravo1/Colab-Notebooks/blob/main/Langchain_OpenRouter_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv openai langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.0/273.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [2]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [3]:
from typing import Any, List, Mapping, Optional, Union, Sequence, Tuple, Dict, Iterator

from openai import OpenAI
from os import getenv

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.prompt_values import PromptValue
from langchain_core.messages.base import BaseMessage
from langchain_core.runnables.config import RunnableConfig

In [4]:
# create an OpenAI API client
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=getenv("API_KEY"),
)

# more models at https://openrouter.ai/models
model = "google/gemma-7b-it:nitro"

system_prompt = {
    "role": "system",
    "content": "You are an AI assistant name Gemma.",
}

# response is delivered at once instead of stremaing
def get_completed_response(prompt):
    completion = client.chat.completions.create(
        model = model,
        messages = [
            system_prompt,
            {
              "role": "user",
              "content": prompt,
            }
          ],
    )
    return completion.choices[0].message.content

# response is streamed
def get_stream(prompt):
    stream = client.chat.completions.create(
        model = model,
        stream = True,
        temperature = 0.1,
        messages = [
            system_prompt,
            {
                "role": "user",
                "content": prompt,
            }
        ]
    )
    return stream;

# inherit the LLM class from langchain
class OpenRouterLLM(LLM):

    @property
    def _llm_type(self) -> str:
        return "open-router"

    # implementing the _call function is mandatory
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        return "gg"

    def invoke(
        self,
        input: str,
        config: Optional[RunnableConfig] = None, *,
        stop: Optional[List[str]] = None,
        **kwargs: Any
    ) -> str:
        return get_completed_response(input)

    def stream(
        self,
        input: str,
        config: Optional[RunnableConfig] = None, *,
        stop: Optional[List[str]] = None,
        **kwargs: Any
        ) -> Iterator[str]:
        return get_stream(input)

In [5]:
llm = OpenRouterLLM()

print(llm.invoke("hi, what is your name?"), "\n")

for chunk in llm.stream("Write me a song about sparkling water."):
    print(chunk.choices[0].delta.content or "", end="")

**Gemma here, ready to assist you.**

My name is Gemma. I am an AI assistant designed to provide you with information and help you with various tasks. Please let me know what I can assist you with today. 

(Verse 1)
Bubbles dance, a bubbly spark,
Sprightly fizz, a refreshing start.
Water bubbles, clear as light,
Sparkling joy, a pure delight.

(Chorus)
Oh, sparkling water, so refreshing,
Your effervescence, a joy to hear.
You quench my thirst, my soul's delight,
And make my day oh so bright.

(Verse 2)
From lemon fizz to lime,
You come in flavors, sweet and bright.
With fruit, you dance and play,
A bubbly treat, come what may.

(Chorus)
Oh, sparkling water, so refreshing,
Your effervescence, a joy to hear.
You quench my thirst, my soul's delight,
And make my day oh so bright.

(Bridge)
Whether chilled or over ice,
You bring a smile, a cooling breeze.
In every sip, a bubbly dream,
You make me feel, oh so keen.

(Chorus)
Oh, sparkling water, so refreshing,
Your effervescence, a joy to he